In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# These will already be installed in the testing environment so disregard the
# amount of time (~1 minute) it takes to install.
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.8/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack ..

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [ ]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 300000

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [ ]:
# Copy one wikidumps files
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP
project_id = 'irfinalproject-415108'
!gcloud config set project {project_id}

data_bucket_name = '318305570_207757535_bucket'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/"


Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists
Copying gs://318305570_207757535_bucket/multistream1_preprocessed.parquet...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1 files][316.7 MiB/316.7 MiB]   23.5 MiB/s                                   
Operation completed over 1 objects/316.7 MiB.                                    


In [ ]:
from pathlib import Path
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
# parquetFile.show()

In [95]:
# take the 'text' and 'id' or the first 1000 rows and create an RDD from it
doc_text_pairs = parquetFile.limit(1000).select("id", "title").rdd
tokens_doc_pairs = doc_text_pairs.map(lambda x: (x[0], pre_calc.tokenize(x[1])))
# def add_ido(pair):
#     if pair[0] == 12 or pair[0] == 25:
#         pair[1].append('ido')
#         pair[1].append('anarchism')
#     return pair

# # Modify the RDD using map
# modified_rdd = tokens_doc_pairs.map(add_ido)
# print(modified_rdd.collect())
# new_tuple = (12, ['anarchism', 'anarchism'])
# tokens_doc_pairs.collect()[0] = new_tuple
# print(tokens_doc_pairs.collect()[0][1])
# tokens = [token.group() for token in RE_WORD.finditer(doc_text_pairs.take(1)[0][1].lower())]
# tokens_doc_pairs = doc_text_pairs.map(lambda x: ([token.group() for token in RE_WORD.finditer(x[0].lower())], x[1]))
# filtered_doc_text_pairs = tokens_doc_pairs.filter(lambda x: "barcelona" in x[0])


In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id, by_id=False):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  word_freq = {}
  for token in text:
      if token not in all_stopwords:
          if token in word_freq:
              word_freq[token] += 1
          else:
              word_freq[token] = 1
  if by_id:
    return [(id, (token, freq)) for token, freq in word_freq.items()]
  else:
    return [(token, (id, freq)) for token, freq in word_freq.items()]



In [92]:
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # Sort the unsorted posting list by wiki_id
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[1], reverse = True)
  return sorted_pl

In [ ]:
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # Count the number of documents each token appears in
  token_df = postings.map(lambda x: (x[0], len(x[1]))).distinct()
  return token_df

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # Partition postings into buckets
  bucketed_postings = postings.groupBy(lambda x: token2bucket_id(x[0]))

  # Write posting lists to disk and get posting locations for each bucket
  posting_locations = bucketed_postings.map(lambda x: InvertedIndex.write_a_posting_list((x[0], list(x[1]))))

  return posting_locations



In [ ]:
# # test partitioning for the different buckets (10 points)
# word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# postings_filtered = postings.filter(lambda x: len(x[1])>10)
# posting_locs_list = partition_postings_and_write(postings_filtered).collect()
# # assert len(posting_locs_list) == 124
# print(posting_locs_list)
# print(len(list(Path('.').glob('*.bin'))))
# # assert len(list(Path('.').glob('*.bin'))) == 124
# pl_lens = [len(pl) for pl in posting_locs_list]
# print(np.max(pl_lens))
# # assert np.max(pl_lens) == 141
# # assert np.min(pl_lens) == 91
# # assert int(np.median(pl_lens)) == 116

In [96]:
word_counts = tokens_doc_pairs.flatMap(lambda x: word_count(x[1], x[0]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# postings_filtered = postings.filter(lambda x: len(x[1])>10)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
posting_locs_list = partition_postings_and_write(postings).collect()
super_posting_locs = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs[k].extend(v)


Putting everything together (posting locations, df) and test that the resulting index is correct.

In [97]:
import math

# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
#partition_func
inverted.posting_locs = super_posting_locs
inverted._posting_list = dict(postings.collect())
##############################################################
##### need to change filtered_doc_text_pairs to doc_text_pairs
##### when testing the whole corpus
##############################################################
# inverted.doc_count = filtered_doc_text_pairs.count()
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Iterate through tokens_doc_pairs to compute TF-IDF for each token in each document
# sum_len_of_all_docs = 0
# for doc_id, words_in_doc in result_rdd.collect():
#   sum_len_docs = 0
#   sum_doc_tf_idf = 0
#   for token, tf in words_in_doc:
#     if inverted.df.get(token) is not None:
#       df = inverted.df[token]
#       idf = math.log(inverted.doc_count / df, 10)
#       sum_len_docs += tf
#       sum_doc_tf_idf += math.pow(tf*idf, 2)
#   inverted.doc_tf_idf[doc_id] = math.sqrt(sum_doc_tf_idf)
#   inverted.doc_len[doc_id] = sum_len_docs
#   sum_len_of_all_docs += sum_len_docs
# inverted.doc_len_avg = sum_len_of_all_docs / inverted.doc_count
# write the global stats out
inverted.write_index('.', 'index_Title')
print(inverted._posting_list)

{'anarchism': [(12, 1)], 'autism': [(25, 1)], 'albedo': [(39, 1)], 'alabama': [(303, 1), (1285, 1), (1286, 1), (1484, 1)], 'achilles': [(305, 1), (2421, 1)], 'abraham': [(307, 1), (1436, 1)], 'lincoln': [(307, 1)], 'aristotle': [(308, 1)], 'american': [(309, 1), (615, 1), (659, 1), (771, 1), (863, 1), (869, 1), (966, 1), (1110, 1), (1111, 1), (1112, 1), (1201, 1), (1241, 1), (1488, 1), (1558, 1), (1890, 1), (1902, 1), (1950, 1), (1973, 1), (2011, 1), (2175, 1), (2357, 1), (2386, 1)], 'paris': [(309, 1)], 'academy': [(316, 1), (324, 1)], 'award': [(316, 1)], 'best': [(316, 1)], 'production': [(316, 1)], 'design': [(316, 1)], 'awards': [(324, 1)], 'actrius': [(330, 1)], 'animalia': [(332, 1)], 'book': [(332, 1), (1726, 1)], 'international': [(334, 1)], 'atomic': [(334, 1), (673, 1), (1194, 1), (1200, 1), (1206, 1), (2014, 1)], 'time': [(334, 1), (1638, 1)], 'altruism': [(336, 1)], 'ayn': [(339, 1)], 'rand': [(339, 1)], 'alain': [(340, 1), (1485, 1)], 'connes': [(340, 1)], 'allan': [(344,

In [ ]:
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [ ]:
pages_links = spark.read.parquet(path).limit(1000).select("id", "anchor_text").rdd

In [ ]:
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
      pages: RDD
          An RDD where each row consists of one Wikipedia article with 'id' and
          'anchor_text'.
  Returns:
  --------
      edges: RDD
          An RDD where each row represents an edge in the directed graph created by
          the Wikipedia links. The first entry should be the source page id and the
          second entry is the destination page id. No duplicates should be present.
      vertices: RDD
          An RDD where each row represents a vertex (node) in the directed graph
          created by the Wikipedia links. No duplicates should be present.
  '''
  # Extract edges from pages
  edges = pages.flatMap(extract_edges).distinct()

  # Extract vertices from edges
  vertices = edges.flatMap(lambda edge: ((edge[0],), (edge[1],))).distinct()

  return edges, vertices

def extract_edges(row):
  ''' Extracts edges from a row of Wikipedia article with 'id' and 'anchor_text'.
  Parameters:
  -----------
      row: tuple
          A tuple representing a row of Wikipedia article with 'id' and 'anchor_text'.
  Returns:
  --------
      edges: list
          A list of tuples representing edges in the directed graph.
  '''
  edges = []
  for anchor_text in row[1]:
      edges.append((row[0], anchor_text[0]))
  return edges

In [ ]:
# test graph construction efficiency (10 points)
t_start = time()
# construct the graph for a small sample of (1000) pages
edges, vertices = generate_graph(pages_links)
# time the actual execution
v_cnt, e_cnt = vertices.count(), edges.count()
graph_const_time = time() - t_start
# test that the implementation is efficient (<20 seconds)
assert graph_const_time < 20

In [ ]:
# test that we get the right number of vertices (5 points)
assert v_cnt == 114966

In [ ]:
# test that we get the right number of edges (5 points)
assert e_cnt == 183727

Let's compute PageRank in a distributed fashion (using PySpark's GraphFrame package) and show the top 20 pages.

In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(4, 'src')
verticesDF = vertices.toDF(['id']).repartition(4, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=10)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv('pr', compression="gzip")
pr.show()

**YOUR TASK (5 POINTS)**: Complete the implmentation of `get_top_pr_page_titles` to return the titles a few of the top pages as ranked by PageRank and printed above. To get the title for each page id listed above, navigate to the url https://en.wikipedia.org/?curid=wiki_id_here by replacing the `wiki_id_here` part with the page id. For example, if you navigate to https://en.wikipedia.org/?curid=25507 for the page with id 25507 you'd see that its title is 'Roman Empire'.

In [ ]:
def get_top_pr_page_titles():
  ''' Returns the title of the first, fourth, and fifth pages as ranked about
      by PageRank.
  Returns:
  --------
    list of three strings.
  '''
  page_titles = ['Catholic Church', 'Ancient Greek', 'Aristotle']
  return page_titles

In [ ]:
assert [_hash(x.lower()) for x in get_top_pr_page_titles()] == \
  ['52bb7dbc02', '3e920ea944', '3d007b19ee']

Do the top ranked pages make sense to you? Why do you think these pages received such a high scores from PageRank?

# GCP

The second part of the assignment involves running index creation in GCP and reporting results in this notebook. Follow [this](https://docs.google.com/document/d/1HTZVDMQ1gyeq8dlyzmomOkxjSo1Td76PsCGlFkQohps/edit?usp=sharing) step-by-step guide in order to set up your cluster environment on GCP so you can execute pyspark code on a cluster of machines.

The code to the GCP notebook is in the moodle under assignment 3. You will have to copy the code you completed here in the relevant sections.

In the current notebook we used MapReduce to create the index for only one wiki dump. Using GCP, we will be able to create the index on the entire english wikipedia. The GCP part of this assignment will be the first time we will work with the entire corpus. You will need to follow the notebook, complete the code as you did here (the same code you completed in this notebook can be used in the GCP notebook).
At the end of the GCP assignment you will have questions that you will need to answer - this are not the tests we used so far, so you should follow the instructions in the notebook.